In [1]:
import numpy as np
import cv2
from skimage.feature import peak_local_max

In [2]:
demOr = cv2.imread('../data/golf_dem.tif', cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)

In [3]:
orthoimage = cv2.imread('../data/golf_ortho.tif')

In [4]:
dem = cv2.GaussianBlur(demOr, (21, 21), 0, 0)

In [5]:
def is_local_maxima(image, row, col):
    ok = True
    if image[row][col] < -30000:
        return False
    for r in range(row-2, row+3):
        for c in range(col-2, col+3):
            ok = ok and image[r][c] <= image[row][col] 
    return ok

In [6]:
def find_local_maxima(image):
    maxima = []
    for i in range(2, image.shape[0] - 2):
        for j in range(2, image.shape[1] - 2):
            if is_local_maxima(image, i, j):
                maxima.append([i, j])
    return np.array(maxima)

In [7]:
loc_max = find_local_maxima(dem)

In [8]:
maxima = peak_local_max(dem, min_distance=15)

In [9]:
def gen_sh(a=1, b=1):
    z = []
    for x in np.linspace(-1.75, 1.75, 15):
        zt = []
        for y in np.linspace(-1.75, 1.75, 15):
            zt.append(- (x ** 2 / a ** 2) - (y ** 2 / b ** 2))
        z.append(zt)
    return np.array(z)

In [10]:
def draw_mark(image, row, col, color):
    for i in range(-2, 3):
        for j in range(-2, 3):
            image[row + i][col + j] = color
    return image

In [11]:
sh = gen_sh(6, 6)
newsh = gen_sh(1.2, 1.2)
newsh -= 5

In [12]:
def substr_mid(image, row, col, sz=4):
    return image[row-sz:row+sz+1, col-sz:col+sz+1] - image[row][col]

In [13]:
tree_count = 0
trees = []
for maxi in loc_max:
    ok = True
    if maxi[0] < 7 or maxi[0] + 7 >= dem.shape[0]:
        continue
    if maxi[1] < 7 or maxi[1] + 7 >= dem.shape[1]:
        continue
    s = substr_mid(dem, maxi[0], maxi[1], 7)
    if (s > sh).any(): # check if neighborhood is lower than some paraboloid
        ok = False
    if (s < newsh).any(): # check that neighborhood is higher than some other paraboloid
        ok = False
    if ok:
        orthoimage = draw_mark(orthoimage, maxi[0], maxi[1], [0, 0, 255])
        tree_count += 1
        trees.append(maxi)
trees = np.array(trees)

In [14]:
trees.shape

(2114, 2)

In [108]:
cv2.imwrite('../data/golf_maxima.jpeg', orthoimage)

True